In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader , PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate , PromptTemplate
from langchain_classic.chains import create_history_aware_retriever , create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain.messages import HumanMessage , AIMessage 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough , RunnableMap


load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

c:\Users\User\Desktop\Ai_Projects\langchain_Updated\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load and split the dataset
loader = TextLoader("langchain_crewai_dataset.txt")
raw_docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300 , chunk_overlap=50)
chunks = text_splitter.split_documents(raw_docs)

In [4]:
#Embed and store in DB

In [3]:
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(chunks , embeddings)


In [5]:
#Retriever MMR
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k":5})

In [6]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000281E18B44D0>, search_type='mmr', search_kwargs={'k': 5})

In [20]:
retriever.invoke("Langchain Models?")

[Document(id='0be58b67-f732-4c44-a21a-8b9894d414b4', metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is compatible with multiple LLM providers including OpenAI, Anthropic, Cohere, Hugging Face, and more. This flexibility ensures that developers can switch between models without rewriting core logic. (v2)'),
 Document(id='a2fb2fad-1175-4e3d-b51f-897717c31e59', metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use'),
 Document(id='9321227d-7e5b-484e-a2fe-40d08f419149', metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is compatible with multiple LLM providers including OpenAI, Anthropic, Cohere, Hugging Face, and more. This f

In [13]:
# LLM and PROMPT
llm = init_chat_model(model="groq:qwen/qwen3-32b" , reasoning_format="hidden")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000281E628DF50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000281E628D690>, model_name='qwen/qwen3-32b', reasoning_format='hidden', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
#Query Expnasion
query_expansion_prompt = PromptTemplate.from_template(
    """You are a helpful assistant.Expand the following query to improve document retrieval 
    by adding relevant synonyms , technical terms , useful context
    
    Original query: "{query}"

    Expanded query: 

    """
)

query_expansion_chain = query_expansion_prompt|llm|StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='You are a helpful assistant.Expand the following query to improve document retrieval \n    by adding relevant synonyms , technical terms , useful context\n\n    Original query: "{query}"\n\n    Expanded query: \n\n    ')
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000281E628DF50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000281E628D690>, model_name='qwen/qwen3-32b', reasoning_format='hidden', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [15]:
query_expansion_chain.invoke({"query":"Langchain Memory"})

'**Expanded Query:**  \n"Langchain Memory" OR "LangChain Memory" OR "Langchain memory framework" OR "LangChain memory module"  \nAND (  \n    "state management" OR "context retention" OR "session memory" OR "conversation history"  \n    OR "memory components" OR "memory modules" OR "persisted memory systems"  \n    OR "LLM memory integration" OR "agent-based memory" OR "long-term memory storage"  \n)  \nAND (  \n    "Langchain framework" OR "LangChain library" OR "Langchain RAG systems" OR "Langchain AI application architecture"  \n    OR "Langchain chatbot memory" OR "Langchain agent memory" OR "Langchain memory optimization"  \n    OR "Langchain memory implementation" OR "Langchain memory patterns"  \n)  \nAND (  \n    "conversation buffer memory" OR "token retention" OR "context window management"  \n    OR "memory persistence" OR "memory caching" OR "memory lifecycle"  \n    OR "Langchain Memory class" OR "Langchain Memory API"  \n)  \nAND (  \n    "AI chatbot development" OR "larg

In [16]:
#RAG ANSWERING PROMPT
answer_prompt = PromptTemplate.from_template(
    """Answer the following question based on the context below
    
    Context: {context}

    Question: {input}
    
    """
)

document_chain = create_stuff_documents_chain(llm=llm , prompt=answer_prompt)

In [17]:
#Full RAG Pipeline
rag_pipeline = (
    RunnableMap({
        "input": lambda x : x["input"],
        "context": lambda x : retriever.invoke(query_expansion_chain.invoke({"query":x["input"]}))
    })| document_chain
)

In [19]:
#Query 
query = {"input": "What types of memory does Langchain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("Answer\n",response)

**Expanded Query:**  
{"input": "What types of memory, storage, or persistence mechanisms does Langchain support? Please include details on memory modules, session handling, caching strategies, state management, or integration with databases (e.g., in-memory databases, key-value stores, SQL/NoSQL). Clarify if Langchain supports transient vs. persistent memory, chatbot session memory, agent memory systems, or specialized memory configurations for tasks like conversational agents, task automation, or multi-turn interactions."}  

**Rationale for Expansion:**  
1. **Synonyms & Broader Terms:** Added "storage," "persistence," and "mechanisms" to capture alternative phrasing.  
2. **Technical Terms:** Included "memory modules," "caching strategies," "state management," and specific database types (in-memory, key-value, SQL/NoSQL) to target technical documentation.  
3. **Use Cases:** Highlighted "chatbot session memory," "agent memory systems," and "multi-turn interactions" to align with La